In [419]:
from bs4 import BeautifulSoup
import requests
import re
from retry import retry 
import pandas as pd
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from copy import deepcopy

if not globals().get("driver"):
    driver = webdriver.Chrome('/Users/ricky/dev/scraping/drivers/chromedriver')

# 必要な関数類
@retry(tries=3, delay=10, backoff=2)
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

def keyword_to_url(keywords):
    first = "https://job.mynavi.jp/24/pc/corpinfo/searchCorpListByGenCond/index?actionMode=searchFw&cond=&"
    last = f"srchWord={keywords}&q={keywords}&SC=corp&srchWordTgt=1"
    return first+last

def make_id_dict(elem_list):
    result = {}
    for item in elem_list:
        _id = re.search("(.*)(_label)", item.get("id"))[1]
        result[item.text] = _id
    return result

def condition_join(words):
    if words:
        return ",".join(words)
    return None

def extract_condition(info_dict):
    conditions = ["検索語", "業種(カテゴリ)", "業種(詳細)", "地域", "職種", "福利厚生", "従業員数"]
    result = []
    for cond in conditions:
        if not info_dict.get(cond):
            result.append(None)
        else:
            result.append(condition_join(info_dict[cond]))
    return result

def modify_condition_str(cond_str):
    return re.search("(.*)(\(\d*件\))", cond_str)[1]

def update_dictkey(target_dict, func):
    _dict={}
    for key, value in target_dict.items():
        _dict[func(key)] = value
    return _dict

def condition_iter(condition, words_list):
    result = [condition, ]
    for words in words_list:
        if type(words) == str:
            words = [words]
        c = copy.copy(condition)
        c["検索語"] = [*words]
        result.append(c)
    return result

def print_search_info(info):
    cond = "None"
    search_word = "None"
    for key, value in info.items():
        if key == "検索語":
            search_word = value
        elif value:
            if cond == "None":
                cond = []
            cond.extend(value)
    print(f"検索語 : {search_word}  条件 : {cond}")
            

# 条件検索のための関数
# dataには条件 + 検索語が入る。条件のフォーマットはsearch_infoを参照
def search(data):
    driver.get("https://job.mynavi.jp/24/pc/corpinfo/displayCorpSearch/index")
    if not data.get("検索語"):
        data["検索語"] = ""
    try:
        check_script = "arguments[0].checked = true"
        for key, values in data.items():
            if key == "検索語":
                continue
            for word in values:
                try:
                    _id = conditions[key][word]
                    _input = driver.find_element(By.ID, _id)
                    driver.execute_script(check_script, _input)
                except:
                    print(f"except : {word}")
        button = driver.find_element(By.ID, "doSearch")
        driver.execute_script("arguments[0].disabled = false", button)
        button.click()

        _input = driver.find_element(By.ID, "srchWord")
        search_words = ",".join(data["検索語"])
        try:
            driver.execute_script(f"arguments[0].value = '{search_words}'", _input)
            button = driver.find_element(By.ID, "doSearch")
            driver.execute_script("arguments[0].click()", button)
        except:
            print("検索結果が少なすぎるため、検索語は無視されます")
        return True
    except BaseException as e:
        print({e})
        return False
    
# 条件検索のフォーマット
# カテゴリごとに条件をリスト形式で入力。サポートされている条件は直下のセルのconditionsを参照
example_info = {
    "業種(カテゴリ)": [], 
    "業種(詳細)" : ['非鉄金属', "金属製品"], 
    "地域": ["東京"], 
    "職種": [], 
    "福利厚生": [],
    "従業員数": [], 
    "検索語" : ["製造"],
}

In [363]:
# 条件の一覧
# 一番下にconditions
if not globals().get("driver"):
    driver = webdriver.Chrome('/Users/ricky/dev/scraping/drivers/chromedriver')
driver.get("https://job.mynavi.jp/24/pc/corpinfo/displayCorpSearch/index")
html = BeautifulSoup(driver.page_source, "html.parser")
ind_category_inputs = html.find_all("a", {"id":re.compile(r"industryCheckLink\d+")})
ind_detail_inputs = html.find_all("a", {"id":re.compile(r"industryCtgDetailedCheckArray\d+")})
area_inputs = html.find_all("a", {"id": re.compile(r"ifRegional\d*")})
occu_inputs = html.find_all("a", {"id": re.compile("occGroup\d+")})
corp_welfare_inputs = html.find_all("label", {"id":re.compile(r"corpWelfare\d+")})
emp_inputs = html.find_all("label", {"id":re.compile(r"empInfo\d+")}) 
job_types = update_dictkey(make_id_dict(occu_inputs), modify_condition_str)

conditions = {
    "業種(カテゴリ)": make_id_dict(ind_category_inputs), 
    "業種(詳細)" : make_id_dict(ind_detail_inputs), 
    "地域": make_id_dict(area_inputs), 
    "職種": job_types, 
    "福利厚生": make_id_dict(corp_welfare_inputs),
    "従業員数": make_id_dict(emp_inputs), 
}

conditions

{'業種(カテゴリ)': {'農林・水産': 'industryCheckLink10',
  '食品': 'industryCheckLink11',
  '建設・設備関連': 'industryCheckLink12',
  '住宅・インテリア': 'industryCheckLink13',
  'アパレル・服飾関連': 'industryCheckLink14',
  '繊維・紙・パルプ': 'industryCheckLink15',
  '化学・石油': 'industryCheckLink16',
  '薬品・化粧品': 'industryCheckLink17',
  'ゴム・ガラス・セラミックス': 'industryCheckLink18',
  '鉄鋼・金属・鉱業': 'industryCheckLink19',
  '機械': 'industryCheckLink110',
  'プラント・エンジニアリング': 'industryCheckLink111',
  '電子・電気・OA機器': 'industryCheckLink112',
  '自動車・輸送用機器': 'industryCheckLink113',
  '精密・医療機器': 'industryCheckLink114',
  '印刷・事務機器・日用品': 'industryCheckLink115',
  'スポーツ・玩具・ゲーム製品': 'industryCheckLink116',
  'その他メーカー・製造関連': 'industryCheckLink117',
  '総合商社': 'industryCheckLink20',
  '商社': 'industryCheckLink21',
  '百貨店・スーパー・コンビニ': 'industryCheckLink30',
  '専門店': 'industryCheckLink31',
  '銀行・証券': 'industryCheckLink40',
  '信金・労金・信組': 'industryCheckLink41',
  'クレジット・信販・リース・その他金融': 'industryCheckLink42',
  '生保・損保': 'industryCheckLink43',
  '不動産': 'industryCh

In [423]:
# driverを使って自動でchromeを操作する。
# コードの使用にはドライバーのインストールが必要 https://chromedriver.chromium.org/downloads
# 以下のコード第一引数はドライバーの絶対パス
driver = webdriver.Chrome('/Users/ricky/dev/scraping/drivers/chromedriver')

/var/folders/9z/_tv0kl191wvbx_078d5dg6fc0000gn/T/ipykernel_58550/1182022750.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/ricky/dev/scraping/drivers/chromedriver')


In [420]:
#★ 使い方
#★ 直下のセルがスクレイピングの本文です。
#★ search_info_list に 検索用のデータをリスト形式で入力すると調査ができます
#★ 検索用のデータフォーマットは以下の example_info の通りです
#★ 条件タイプ : [条件1, 条件2, ...] のフォーマットで条件を入力します。
#★ 使っていない条件は[]でもいいですし("職種"欄のように)、そもそも含めなくてもいいです。"従業員数"は含まれていませんがエラーにはなりません
#★ また、データの最後に検索語をつけます

example_info = {
    "業種(カテゴリ)": [], 
    "業種(詳細)" : ['非鉄金属', "金属製品"], 
    "地域": ["東京"], 
    "職種": [], 
    "福利厚生": [],
    #"従業員数": "なしでもいい", 
    "検索語" : ["製造"],
}



#★ 使い方2
#★ 何度も同じ条件で検索語のみを変えて検索する際は、いちいちフォーマットを作るのが面倒になります
#★ そこで　condition_iter 関数を使います
#★ 第1引数に条件を第２引数に検索語の「リストのリスト」を渡します。つまり、以下の通りです。

first_param = {
    "業種(詳細)" : ['非鉄金属'], 
    "地域": ["宮城"],
}

second_param = [("製造", "金属"), ("メーカー"), ("食品", "メーカー"), ("海外赴任")] 

#★ 第1引数は条件です。少し単純にしました。
#★ 第2引数は検索語の「リストのリスト」です。ここではわかりやすく、内側をタプルにしました。
#★ これらを condition_iter 関数に渡すと、条件 + それぞれの検索語セットのデータをリスト形式で渡してくれます
#★ リスト同士は extend() を使えば繋げられるので、list_1.extend(list_2) で検索データのリストをつなげたりして使ったりできます

condition_iter(first_param, second_param)


[{'業種(詳細)': ['非鉄金属'], '地域': ['宮城']},
 {'業種(詳細)': ['非鉄金属'], '地域': ['宮城'], '検索語': ['製造', '金属']},
 {'業種(詳細)': ['非鉄金属'], '地域': ['宮城'], '検索語': ['メーカー']},
 {'業種(詳細)': ['非鉄金属'], '地域': ['宮城'], '検索語': ['食品', 'メーカー']},
 {'業種(詳細)': ['非鉄金属'], '地域': ['宮城'], '検索語': ['海外赴任']}]

In [421]:
# スクレイピング
# 条件と検索語をsearch_info_listにリスト形式で入れる
search_info_list = [example_info]

data = []

for info in search_info_list:
    print("=====================================================================")
    print_search_info(info)
    success = search(info)
    if not success:
        continue
    result_num = int(driver.find_element(By.ID, "searchResultkensuu").text)
    print(f"Search results number : {result_num}")
    next_page = True
    found = False
    page = 0
    while next_page and not found:
        print(f"Searching... page {page}")
        html = BeautifulSoup(driver.page_source, "html.parser")
        a_tags = html.find_all("a", {"id":re.compile("corpNameLink\[\d+\]")})
        for num, a in enumerate(a_tags):
            if a.text == "タキゲン製造(株)":
                order = int(page * 100 + num + 1)
                row = [*extract_condition(info), result_num, True, order, round(order/result_num, 4)*100]
                print(f"New data gained : {row}")
                data.append(row)
                found = True
                break
        try:
            next_button = driver.find_element(By.ID, "upperNextPage")
            next_button.click()
            page += 1
        except:
            next_page = False
    if not found:
        row = [*extract_condition(info), result_num, False, None, None]
        print(f"New data gained : {row}")
        data.append(row)

columns=["検索語", "業種(カテゴリ)", "業種(詳細)", "地域", "職種", "福利厚生", "従業員数", "検索結果数", "結果の有無", "順位", "検索結果の上位%"]
df = pd.DataFrame(data, columns=columns)

検索語 : None  条件 : ['非鉄金属', '宮城']
Search results number : 5
Searching... page 0
New data gained : [None, None, '非鉄金属', '宮城', None, None, None, 5, False, None, None]
検索語 : ['製造', '金属']  条件 : ['非鉄金属', '宮城']
Search results number : 1
Searching... page 0
New data gained : ['製造,金属', None, '非鉄金属', '宮城', None, None, None, 1, False, None, None]
検索語 : ['メーカー']  条件 : ['非鉄金属', '宮城']
Search results number : 2
Searching... page 0
New data gained : ['メーカー', None, '非鉄金属', '宮城', None, None, None, 2, False, None, None]
検索語 : ['食品', 'メーカー']  条件 : ['非鉄金属', '宮城']
Search results number : 0
Searching... page 0
New data gained : ['食品,メーカー', None, '非鉄金属', '宮城', None, None, None, 0, False, None, None]
検索語 : ['海外赴任']  条件 : ['非鉄金属', '宮城']
Search results number : 0
Searching... page 0
New data gained : ['海外赴任', None, '非鉄金属', '宮城', None, None, None, 0, False, None, None]


In [422]:
df

,検索語,業種(カテゴリ),業種(詳細),地域,職種,福利厚生,従業員数,検索結果数,結果の有無,順位,検索結果の上位%
0,None,None,非鉄金属,宮城,None,None,None,5,False,None,None
1,"製造,金属",None,非鉄金属,宮城,None,None,None,1,False,None,None
2,メーカー,None,非鉄金属,宮城,None,None,None,2,False,None,None
3,"食品,メーカー",None,非鉄金属,宮城,None,None,None,0,False,None,None
4,海外赴任,None,非鉄金属,宮城,None,None,None,0,False,None,None
